In [1]:
import pandas as pd
import numpy as np

from google.colab import drive

# Bibliotecas de pré-processamento
from sklearn.preprocessing import LabelEncoder

# Bibliotecas de plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import RidgeClassifierCV, LogisticRegression, PassiveAggressiveClassifier, SGDClassifier
from sklearn.svm import LinearSVC, SVC, NuSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Data/merged_df.csv')

In [4]:
from sklearn.preprocessing import StandardScaler

df = df.drop(['record_id', 'idade_faixa'], axis=1)

X_afetam_eficacia = df[['idade_diagnostico','primeiro_IMC','ultima_informacao_paciente', 'tempo_seguimento', 'recidiva_distancia', 'recidiva_regional', 'recidiva_local']]
X_afetam_eficacia[['tempo_seguimento']] = StandardScaler().fit_transform(X_afetam_eficacia[['tempo_seguimento']])

<ipython-input-4-6de22f19c6fc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_afetam_eficacia[['tempo_seguimento']] = StandardScaler().fit_transform(X_afetam_eficacia[['tempo_seguimento']])


In [5]:
km = KMeans( n_clusters = 2, init = 'random', max_iter = 300, n_init = 100, random_state = 52 )
km.fit( X_afetam_eficacia )
y_km = km.predict( X_afetam_eficacia )
df['eficacia_tratamento'] = y_km

In [6]:
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 1")))

# Este é um sistema de classificação de tratamento em que "0" significa que o tratamento foi bom e "1" significa 
# que foi ruim. Há dois casos apresentados: no primeiro, uma pessoa idosa de 80 anos sobreviveu ao tratamento, indicando 
# que ele foi bom. No segundo caso, não houve sobreviventes entre as pessoas mais velhas e, portanto, o tratamento foi classificado 
# como negativo para essa faixa etária. É importante lembrar que a eficácia do tratamento pode variar de acordo com vários fatores e 
# que cada caso é único.

0
38


In [7]:
df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")

print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")))

# Este modelo classifica o tratamento como bom (0) ou ruim (1). Ele foi testado em dois casos de mulheres com menos
# de 30 anos: uma morreu e a outra sobreviveu. O modelo foi capaz de classificar corretamente cada caso, indicando que 
# tem uma boa capacidade de classificação.

12
0


In [8]:
df = df.query("tratamento == 0 or tratamento == 1")[:]

In [9]:
import numpy as np

df = df.astype('float64')
df = df[np.isfinite(df).all(1)]

In [10]:
# Divisao do conjunto Teste/Treino
from sklearn.model_selection import train_test_split

X = df.drop(['tratamento'], axis=1)
y = df['tratamento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 73)

In [11]:
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:22<00:00,  1.29it/s]


In [13]:
# Metricas antes do ajuste dos hiperparametros
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.67,0.67,0.67,0.67,0.38
RandomForestClassifier,0.67,0.67,0.67,0.67,1.34
LGBMClassifier,0.67,0.66,0.66,0.66,2.51
ExtraTreesClassifier,0.66,0.66,0.66,0.66,1.26
LinearSVC,0.66,0.66,0.66,0.66,0.59
RidgeClassifierCV,0.66,0.66,0.66,0.66,0.08
LogisticRegression,0.66,0.66,0.66,0.66,0.08
SVC,0.66,0.66,0.66,0.66,0.81
LinearDiscriminantAnalysis,0.66,0.65,0.65,0.66,0.13


In [14]:
classifiers = [AdaBoostClassifier(), 
               RandomForestClassifier(),
               ExtraTreesClassifier(),
               RidgeClassifierCV(), 
               LinearSVC(),
               LogisticRegression(), 
               SVC(), 
               LGBMClassifier(),
               LinearDiscriminantAnalysis(), 
               RidgeClassifier(), 
               CalibratedClassifierCV(), 
               XGBClassifier(),
               NearestCentroid(), 
               SGDClassifier(), 
               BaggingClassifier(),
               BernoulliNB(),
               PassiveAggressiveClassifier(), 
               Perceptron(),
               NuSVC(), 
               DecisionTreeClassifier(), 
               KNeighborsClassifier(),
               LabelSpreading(),
               LabelPropagation(), 
               ExtraTreeClassifier(),
               GaussianNB(),
               DummyClassifier(),
               QuadraticDiscriminantAnalysis()]

precisions = []
recalls = []

for classifier in classifiers:
    clf = classifier.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    precisions.append(precision_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred))

In [15]:
models['Precision'] = 0
models['Recall'] = 0   

for i in range(0, len(precisions)):
  models['Precision'].iloc[i] = precisions[i]
  models['Recall'].iloc[i] = recalls[i]

models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken,Precision,Recall
Model,,,,,,,
AdaBoostClassifier,0.67,0.67,0.67,0.67,0.38,0.66,0.62
RandomForestClassifier,0.67,0.67,0.67,0.67,1.34,0.67,0.59
LGBMClassifier,0.67,0.66,0.66,0.66,2.51,0.66,0.62
ExtraTreesClassifier,0.66,0.66,0.66,0.66,1.26,0.65,0.60
LinearSVC,0.66,0.66,0.66,0.66,0.59,0.47,1.00
RidgeClassifierCV,0.66,0.66,0.66,0.66,0.08,0.63,0.60
LogisticRegression,0.66,0.66,0.66,0.66,0.08,0.00,0.00
SVC,0.66,0.66,0.66,0.66,0.81,0.65,0.60
LinearDiscriminantAnalysis,0.66,0.65,0.65,0.66,0.13,0.64,0.59


# Accuracy: 0.6641550053821313
# Balanced Accuracy: 0.6591726283627692
# ROC AUC: 0.6591726283627691
# F1 Score: 0.6622626319221622

In [16]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6716899892357373

In [17]:
X_train

,escolaridade,idade_diagnostico,sexo,raca,ultima_informacao_paciente,tempo_seguimento,ja_gravida,gestacao_idade,tempo_amamentacao,grau_parentesco_mama_primeiro_1_vez,...,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC,diferenca_peso,coef_peso,diferenca_tempo,coef_peso_tempo,eficacia_tratamento
2887,3.00,32.00,0.00,1.00,3.00,1089.00,1.00,25.00,18.00,0.00,...,153.00,32.04,82.95,153.00,35.44,7.95,1.11,994.00,0.01,0.00
1763,5.00,56.00,0.00,3.00,3.00,1368.00,1.00,30.00,8.00,0.00,...,168.00,27.25,87.08,168.00,30.85,2.23,1.13,2213.00,0.00,1.00
2735,5.00,64.00,0.00,0.00,3.00,1182.00,1.00,22.00,50.00,0.00,...,155.00,26.53,45.37,155.00,18.88,-15.96,1.17,1419.00,0.00,1.00
2224,5.00,52.00,0.00,4.00,0.00,1515.00,1.00,28.00,34.00,0.00,...,145.00,27.28,64.00,155.00,26.64,6.65,1.12,1443.00,0.00,0.00
2893,3.00,56.00,0.00,0.00,3.00,1060.00,1.00,26.00,18.00,0.00,...,167.00,32.38,95.30,167.00,34.17,5.00,1.06,1454.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,2.00,59.00,0.00,0.00,3.00,3190.00,1.00,29.00,43.00,0.00,...,168.00,24.80,83.80,168.00,29.69,13.80,1.20,2620.00,0.01,1.00
1449,2.00,38.00,0.00,1.00,1.00,557.00,1.00,19.00,31.00,0.00,...,160.50,21.23,58.75,160.50,22.81,4.05,1.07,203.00,0.02,0.00
405,3.00,71.00,0.00,4.00,0.00,950.00,1.00,20.00,63.00,0.00,...,160.00,23.05,50.40,157.00,20.45,-8.60,0.85,976.00,-0.01,1.00
151,1.00,52.00,0.00,1.00,0.00,1597.00,1.00,25.00,27.00,0.00,...,160.00,24.18,63.00,159.00,24.92,1.10,1.02,1110.00,0.00,0.00


In [22]:
colunas = X_train.columns
X_run = pd.DataFrame(columns=colunas)

In [24]:
colunas

Index(['escolaridade', 'idade_diagnostico', 'sexo', 'raca',
       'ultima_informacao_paciente', 'tempo_seguimento', 'ja_gravida',
       'gestacao_idade', 'tempo_amamentacao',
       'grau_parentesco_mama_primeiro_1_vez',
       'grau_parentesco_mama_primeiro_mais_vezes',
       'grau_parentesco_mama_segundo_1_vez',
       'grau_parentesco_mama_segundo_mais_vezes', 'idade_primeira_menstruacao',
       'uso_anticoncepcional', 'atividade_fisica', 'consumo_alcool',
       'grau_parentesco_primeiro', 'grau_parentesco_segundo',
       'grau_parentesco_terceiro', 'anti_her2_neoadjuvante',
       'primeiro_diagnostico', 'grau_histologico', 'subtipo_tumoral',
       'receptor_estrogenio', 'receptor_progesterona', 'ki67',
       'Receptor de progesterona (quantificação %)',
       'Receptorde Estrogênio (quantificação %)',
       'Indice H (Receptorde progesterona)', 'her2_ihc', 'her2_fish',
       'Ki67 (%)', 'cid_o', 'morfologia_cid_o', 'estadio_clinico',
       'grupo_estadio_clinico', 'cla

In [23]:
X_run
nova_linha = {'escolaridade': , 'idade_diagnostico', 'sexo', 'raca',
       'ultima_informacao_paciente', 'tempo_seguimento', 'ja_gravida',
       'gestacao_idade', 'tempo_amamentacao',
       'grau_parentesco_mama_primeiro_1_vez',
       'grau_parentesco_mama_primeiro_mais_vezes',
       'grau_parentesco_mama_segundo_1_vez',
       'grau_parentesco_mama_segundo_mais_vezes', 'idade_primeira_menstruacao',
       'uso_anticoncepcional', 'atividade_fisica', 'consumo_alcool',
       'grau_parentesco_primeiro', 'grau_parentesco_segundo',
       'grau_parentesco_terceiro', 'anti_her2_neoadjuvante',
       'primeiro_diagnostico', 'grau_histologico', 'subtipo_tumoral',
       'receptor_estrogenio', 'receptor_progesterona', 'ki67',
       'Receptor de progesterona (quantificação %)',
       'Receptorde Estrogênio (quantificação %)',
       'Indice H (Receptorde progesterona)', 'her2_ihc', 'her2_fish',
       'Ki67 (%)', 'cid_o', 'morfologia_cid_o', 'estadio_clinico',
       'grupo_estadio_clinico', 'classificacao_tnm_t', 'classificacao_tnm_n',
       'classificacao_tnm_m', 'combinacao_tratamentos', 'ano_diagnostico',
       'lateralidade_tumor', 'descricao_morfologia', 'descricao_topografia',
       'recidiva_distancia', 'recidiva_regional', 'recidiva_local',
       'primeiro_peso', 'primeira_altura', 'primeiro_IMC', 'ultimo_peso',
       'ultima_altura', 'ultimo_IMC', 'diferenca_peso', 'coef_peso',
       'diferenca_tempo', 'coef_peso_tempo', 'eficacia_tratamento'}

,escolaridade,idade_diagnostico,sexo,raca,ultima_informacao_paciente,tempo_seguimento,ja_gravida,gestacao_idade,tempo_amamentacao,grau_parentesco_mama_primeiro_1_vez,...,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC,diferenca_peso,coef_peso,diferenca_tempo,coef_peso_tempo,eficacia_tratamento


##GridSearch

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Grid Search do modelo AdaBoostClasifier

model = AdaBoostClassifier()

# Escolha dos hiperparametros
params = {
          'n_estimators': [50, 100, 150],
          'learning_rate': [0.01, 0.1],
          'algorithm': ['SAMME', 'SAMME.R'],
           'random_state': [73]
          }

grid_searchAda = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_searchAda.fit(X_train, y_train)

print(grid_searchAda.best_params_)

# {'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150, 'random_state': 73}

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150, 'random_state': 73}
0.6837443194516875
0.673842841765339


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = grid_searchAda.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.673842841765339
# Balanced Accuracy: 0.6680755831284
# ROC AUC: 0.6680755831284001
# F1 Score: 0.6713062718325303

Accuracy: 0.673842841765339
Balanced Accuracy: 0.6680755831284
ROC AUC: 0.6680755831284001
F1 Score: 0.6713062718325303


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Grid Search do modelo RandomForestClasifier

model = RandomForestClassifier()

# Escolha dos hiperparametros
params = {
          'n_estimators': [50, 100, 150],
          'max_depth': [5,10,15],
          'max_features':['sqrt','log2', None],
          'random_state': [73]
          }

grid_searchRF = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_searchRF.fit(X_train, y_train)

print(grid_searchRF.best_params_)

# {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100, 'random_state': 73}

{'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100, 'random_state': 73}
0.6911463266674472
0.6598493003229279


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = grid_searchRF.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.6598493003229279
# Balanced Accuracy: 0.6543916275430359
# ROC AUC: 0.6543916275430359
# F1 Score: 0.6575477826730564

Accuracy: 0.6598493003229279
Balanced Accuracy: 0.6543916275430359
ROC AUC: 0.6543916275430359
F1 Score: 0.6575477826730564


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier()

# Escolha dos parametros
params = {
    'max_depth': [5, 10],
    'learning_rate': [0.01, 0.05],
    'n_estimators': [50, 100],
    'random_state': [73]
}

grid_searchLGBM = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_searchLGBM.fit(X_train, y_train)

print(grid_searchLGBM.best_params_)

# {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 50, 'random_state': 73}

{'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 50, 'random_state': 73}
0.6842125988442014
0.6727664155005382


In [ ]:
# Grid Search do modelo LGBM


from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = grid_searchLGBM.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.6727664155005382
# Balanced Accuracy: 0.667523660481407
# ROC AUC: 0.667523660481407
# F1 Score: 0.6706789482402674

Accuracy: 0.6727664155005382
Balanced Accuracy: 0.667523660481407
ROC AUC: 0.667523660481407
F1 Score: 0.6706789482402674


##RandomizedSearchCV

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

model = AdaBoostClassifier()

# Escolha dos parametros
params = {
          'n_estimators': [50, 100, 150],
          'learning_rate': [0.01, 0.1],
          'algorithm': ['SAMME', 'SAMME.R'],
           'random_state': [73]
          }

randomizedSearchAda = RandomizedSearchCV(estimator=model, param_distributions=params, cv=5, n_iter=50)

randomizedSearchAda.fit(X_train, y_train)

print(randomizedSearchAda.best_params_)
# {'random_state': 73, 'n_estimators': 150, 'learning_rate': 0.1, 'algorithm': 'SAMME.R'}

{'random_state': 73, 'n_estimators': 150, 'learning_rate': 0.1, 'algorithm': 'SAMME.R'}
0.6837443194516875
0.673842841765339


In [ ]:
# Random Search do modelo AdaBoostClasifier

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = randomizedSearchAda.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

# imprimindo as métricas
print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.673842841765339
# Balanced Accuracy: 0.6680755831284
# ROC AUC: 0.6680755831284001
# F1 Score: 0.6713062718325303

Accuracy: 0.673842841765339
Balanced Accuracy: 0.6680755831284
ROC AUC: 0.6680755831284001
F1 Score: 0.6713062718325303


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

# Escolha dos hiperparametros
random_grid = {
          'n_estimators': [50, 100, 150],
          'max_depth': [5,10,15],
          'max_features':['sqrt','log2', None],
          'random_state': [73]
               }

randomizedSearchRF = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

randomizedSearchRF.fit(X_train, y_train)

print(randomizedSearchRF.best_params_)

# {'random_state': 73, 'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 10}

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'random_state': 73, 'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 10}
0.6809787626962143
0.6641550053821313


In [ ]:
# Random Search do modelo RandomForestClasifier

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = randomizedSearchRF.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.6641550053821313
# Balanced Accuracy: 0.6591726283627692
# ROC AUC: 0.6591726283627691
# F1 Score: 0.6622626319221622

Accuracy: 0.6641550053821313
Balanced Accuracy: 0.6591726283627692
ROC AUC: 0.6591726283627691
F1 Score: 0.6622626319221622


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier()

# Escolha dos hiperparametros
params = {
    'max_depth': [5, 10],
    'learning_rate': [0.01, 0.05],
    'n_estimators': [50, 100],
    'random_state': [73]
}

randomizedSearchLGBM = RandomizedSearchCV(estimator=model, param_distributions=params, cv=5)

randomizedSearchLGBM.fit(X_train, y_train)

print(randomizedSearchLGBM.best_params_)
print(randomizedSearchLGBM.best_score_)

{'random_state': 73, 'n_estimators': 50, 'max_depth': 10, 'learning_rate': 0.05}
0.6842125988442014
0.6727664155005382


In [ ]:
# Random Search do modelo RandomForestClasifier

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = randomizedSearchLGBM.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.6727664155005382
# Balanced Accuracy: 0.667523660481407
# ROC AUC: 0.667523660481407
# F1 Score: 0.6706789482402674

Accuracy: 0.6727664155005382
Balanced Accuracy: 0.667523660481407
ROC AUC: 0.667523660481407
F1 Score: 0.6706789482402674


In [ ]:
data_random = {'modelos': ['AdaBoost', 'RandomForest', 'LGBM'],
               'valores': [0.683, 0.680, 0.684],
               'valores_antigos': [0.673, 0.664, 0.672]}

df = pd.DataFrame(data_random)

df = df.rename(columns={'valores_antigos': 'Acurácia antiga', 'valores': 'Acurácia Nova'})

df['tamanho'] = df['Acurácia Nova']

fig = px.bar(df, x='modelos', y=['Acurácia antiga', 'Acurácia Nova'],
             labels={'Acurácia antiga': 'Acurácia antiga', 'Acurácia Nova': 'Acurácia Nova'},
             color_discrete_sequence=px.colors.qualitative.Pastel,
             title='Acurácia Sem Random Search / Acurácia Com Random Search',
             barmode='group')

fig.update_traces(texttemplate='%{y:.3f}', textposition='outside')
fig.show()

In [ ]:
data = {'modelos': ['AdaBoost', 'RandomForest', 'LGBM'], 
        'valores': [0.683, 0.691, 0.684],
        'valores_antigos': [0.673, 0.659, 0.672]}

df = pd.DataFrame(data)

df = df.rename(columns={'valores_antigos': 'Acurácia antiga', 'valores': 'Acurácia Nova'})

fig = px.bar(df, x='modelos', y=['Acurácia antiga', 'Acurácia Nova'],
             labels={'Acurácia antiga': 'Acurácia antiga', 'Acurácia Nova': 'Acurácia Nova'},
             color_discrete_sequence=px.colors.qualitative.Pastel,
             title='Acurácia Sem Grid Search / Acurácia Com Grid Search',
             barmode='group')

fig.update_traces(texttemplate='%{y:.3f}', textposition='outside')

fig.show()

##Conclusão

Logo apartir destas métricas o melhor modelo é o RandomForestClassifier com 69.1% de acurácia utilizando estes parametros: 
{'n_estimators': [50, 100, 150], 'max_depth': [5,10,15], 'max_features':['sqrt','log2', None], 'random_state': [73]}


##Interface para Usúarios

In [26]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:

In [36]:
import gradio

#fazer com dropdown as opções categoricas, transformar tudo em dropdown com entrada normal, que dentro da função transforme por exemplo:
# entra como Fundamental Incompleto -> 3 

inputs = [
          gradio.inputs.Dropdown(['Analfabeto', 'Fundamental incompleto', 'Fundamental completo', 'Médio incompleto', 'Médio completo', 'Superior incompleto', 'Superior completo'], label='Escolaridade'),
          gradio.inputs.Number(label='Idade do diagnóstico'),
          gradio.inputs.Radio(['Masculino', 'Feminino'], label='Sexo'),
          gradio.inputs.Dropdown(['Branca', 'Preta', 'Parda', 'Amarela', 'Indígena'], label='Raça'),
          gradio.inputs.Textbox(label='Última informação do paciente'),
          gradio.inputs.Number(label='Tempo de seguimento'),
          gradio.inputs.Radio(['Sim', 'Não'], label='Já esteve grávida?'),
          gradio.inputs.Number(label='Idade na primeira gestação'),
          gradio.inputs.Number(label='Tempo de amamentação (em meses)'),
          gradio.inputs.Dropdown(['Pai', 'Mãe', 'Irmão', 'Irmã', 'Filho', 'Filha', 'Avô', 'Avó', 'Tio', 'Tia', 'Primo', 'Prima', 'Outro'], label='Grau de parentesco do(a) primeiro(a) parente diagnosticado(a) com câncer de mama (1ª vez)'),
          gradio.inputs.Dropdown(['Pai', 'Mãe', 'Irmão', 'Irmã', 'Filho', 'Filha', 'Avô', 'Avó', 'Tio', 'Tia', 'Primo', 'Prima', 'Outro'], label='Grau de parentesco do(a) primeiro(a) parente diagnosticado(a) com câncer de mama (mais de uma vez)'),
          gradio.inputs.Dropdown(['Pai', 'Mãe', 'Irmão', 'Irmã', 'Filho', 'Filha', 'Avô', 'Avó', 'Tio', 'Tia', 'Primo', 'Prima', 'Outro'], label='Grau de parentesco do(a) segundo(a) parente diagnosticado(a) com câncer de mama (1ª vez)'),
          gradio.inputs.Dropdown(['Pai', 'Mãe', 'Irmão', 'Irmã', 'Filho', 'Filha', 'Avô', 'Avó', 'Tio', 'Tia', 'Primo', 'Prima', 'Outro'], label='Grau de parentesco do(a) segundo(a) parente diagnosticado(a) com câncer de mama (mais de uma vez)'),
          gradio.inputs.Number(label='Idade na primeira menstruação'),
          gradio.inputs.Radio(['Sim', 'Não'], label='Já usou anticoncepcional?'),
          gradio.inputs.Radio(['Sim', 'Não'], label='Faz atividade física regularmente?'),
          gradio.inputs.Radio(['Sim', 'Não'], label='Consome álcool regularmente?'),
          gradio.inputs.Dropdown(['Pai', 'Mãe', 'Irmão', 'Irmã', 'Filho', 'Filha', 'Avô', 'Avó', 'Tio', 'Tia', 'Primo', 'Prima', 'Outro'], label='Grau de parentesco do primeiro parente com câncer de mama'),
          gradio.inputs.Dropdown(['Pai', 'Mãe', 'Irmão', 'Irmã', 'Filho', 'Filha', 'Avô', 'Avó', 'Tio', 'Tia', 'Primo', 'Prima', 'Outro'], label='Grau de parentesco do segundo parente com câncer de mama'),
          gradio.inputs.Dropdown(['Pai', 'Mãe', 'Irmão', 'Irmã', 'Filho', 'Filha', 'Avô', 'Avó', 'Tio', 'Tia', 'Primo', 'Prima', 'Outro'], label='Grau de parentesco do terceiro parente com câncer de mama'),
          gradio.inputs.Radio(['Sim', 'Não'], label='Já fez uso de anti-HER2 neoadjuvante?'),
          gradio.inputs.Textbox(label='Descrição do primeiro diagnóstico'),
          gradio.inputs.Textbox(label='grau_histologico'),
          gradio.inputs.Textbox(label='subtipo_tumoral'),
          gradio.inputs.Textbox(label='receptor_estrogenio'),
          gradio.inputs.Textbox(label='receptor_progesterona'),
          gradio.inputs.Textbox(label='ki67'),
          gradio.inputs.Textbox(label='Receptor_de_progesterona_quantificacao'),
          gradio.inputs.Textbox(label='Receptorde_estrogenio_quantificacao'),
          gradio.inputs.Textbox(label='Indice_H_Receptorde_progesterona'),
          gradio.inputs.Textbox(label='her2_ihc'),
          gradio.inputs.Textbox(label='her2_fish'),
          gradio.inputs.Textbox(label='Ki67'),
          gradio.inputs.Textbox(label='cid_o'),
          gradio.inputs.Textbox(label='morfologia_cid_o'),
          gradio.inputs.Textbox(label='estadio_clinico'),
          gradio.inputs.Textbox(label='grupo_estadio_clinico'),
          gradio.inputs.Textbox(label='classificacao_tnm_t'),
          gradio.inputs.Textbox(label='classificacao_tnm_n'),
          gradio.inputs.Textbox(label='classificacao_tnm_m'),
          gradio.inputs.Textbox(label='combinacao_tratamentos'),
          gradio.inputs.Textbox(label='ano_diagnostico'),
          gradio.inputs.Textbox(label='lateralidade_tumor'),
          gradio.inputs.Textbox(label='descricao_morfologia'),
          gradio.inputs.Textbox(label='descricao_topografia'),
          gradio.inputs.Textbox(label='recidiva_distancia'),
          gradio.inputs.Textbox(label='recidiva_regional'),
          gradio.inputs.Textbox(label='recidiva_local'),
          gradio.inputs.Textbox(label='primeiro_peso'),
          gradio.inputs.Textbox(label='primeira_altura'),
          gradio.inputs.Textbox(label='primeiro_IMC'),
          gradio.inputs.Textbox(label='ultimo_peso'),
          gradio.inputs.Textbox(label='ultima_altura'),
          gradio.inputs.Textbox(label='ultimo_IMC'),
          gradio.inputs.Textbox(label='diferenca_peso'),
          gradio.inputs.Textbox(label='coef_peso'),
          gradio.inputs.Textbox(label='diferenca_tempo'),
          gradio.inputs.Textbox(label='coef_peso_tempo'),
          gradio.inputs.Textbox(label='eficacia_tratamento'),
]

output = gradio.outputs.Label(label='Resultado da predição')

def runModel(escolaridade, idade_diagnostico, sexo, raca,
       ultima_informacao_paciente, tempo_seguimento, ja_gravida,
       gestacao_idade, tempo_amamentacao,
       grau_parentesco_mama_primeiro_1_vez,
       grau_parentesco_mama_primeiro_mais_vezes,
       grau_parentesco_mama_segundo_1_vez,
       grau_parentesco_mama_segundo_mais_vezes, idade_primeira_menstruacao,
       uso_anticoncepcional, atividade_fisica, consumo_alcool,
       grau_parentesco_primeiro, grau_parentesco_segundo,
       grau_parentesco_terceiro, anti_her2_neoadjuvante,
       primeiro_diagnostico, grau_histologico, subtipo_tumoral,
       receptor_estrogenio, receptor_progesterona, ki67,
       Receptor_de_progesterona_quantificacao,
       Receptorde_estrogenio_quantificacao,
       Indice_H_Receptorde_progesterona, her2_ihc, her2_fish,
       Ki67, cid_o, morfologia_cid_o, estadio_clinico,
       grupo_estadio_clinico, classificacao_tnm_t, classificacao_tnm_n,
       classificacao_tnm_m, combinacao_tratamentos, ano_diagnostico,
       lateralidade_tumor, descricao_morfologia, descricao_topografia,
       recidiva_distancia, recidiva_regional, recidiva_local,
       primeiro_peso, primeira_altura, primeiro_IMC, ultimo_peso,
       ultima_altura, ultimo_IMC, diferenca_peso, coef_peso,
       diferenca_tempo, coef_peso_tempo, eficacia_tratamento):
  
  nova_linha = {'escolaridade': escolaridade, 'idade_diagnostico': idade_diagnostico, 'sexo': sexo, 'raca': raca,
       'ultima_informacao_paciente' : ultima_informacao_paciente, 'tempo_seguimento': tempo_seguimento, 'ja_gravida': ja_gravida,
       'gestacao_idade': gestacao_idade, 'tempo_amamentacao': tempo_amamentacao,
       'grau_parentesco_mama_primeiro_1_vez': grau_parentesco_mama_primeiro_1_vez,
       'grau_parentesco_mama_primeiro_mais_vezes': grau_parentesco_mama_primeiro_mais_vezes,
       'grau_parentesco_mama_segundo_1_vez': grau_parentesco_mama_segundo_1_vez,
       'grau_parentesco_mama_segundo_mais_vezes': grau_parentesco_mama_segundo_mais_vezes, 'idade_primeira_menstruacao': idade_primeira_menstruacao,
       'uso_anticoncepcional': uso_anticoncepcional, 'atividade_fisica': atividade_fisica, 'consumo_alcool': consumo_alcool,
       'grau_parentesco_primeiro': grau_parentesco_primeiro, 'grau_parentesco_segundo': grau_parentesco_segundo,
       'grau_parentesco_terceiro': grau_parentesco_terceiro, 'anti_her2_neoadjuvante': anti_her2_neoadjuvante,
       'primeiro_diagnostico': primeiro_diagnostico, 'grau_histologico': grau_histologico, 'subtipo_tumoral': subtipo_tumoral,
       'receptor_estrogenio': receptor_estrogenio, 'receptor_progesterona': receptor_progesterona, 'ki67': ki67,
       'Receptor de progesterona (quantificação %)': Receptor_de_progesterona_quantificacao,
       'Receptorde Estrogênio (quantificação %)': Receptorde_estrogenio_quantificacao,
       'Indice H (Receptorde progesterona)': Indice_H_Receptorde_progesterona, 'her2_ihc': her2_ihc, 'her2_fish': her2_fish,
       'Ki67 (%)': Ki67, 'cid_o': cid_o, 'morfologia_cid_o': morfologia_cid_o, 'estadio_clinico': estadio_clinico,
       'grupo_estadio_clinico': grupo_estadio_clinico, 'classificacao_tnm_t': classificacao_tnm_t, 'classificacao_tnm_n': classificacao_tnm_n,
       'classificacao_tnm_m': classificacao_tnm_m, 'combinacao_tratamentos': combinacao_tratamentos, 'ano_diagnostico': ano_diagnostico,
       'lateralidade_tumor': lateralidade_tumor, 'descricao_morfologia': descricao_morfologia, 'descricao_topografia': descricao_topografia,
       'recidiva_distancia': recidiva_distancia, 'recidiva_regional': recidiva_regional, 'recidiva_local': recidiva_local,
       'primeiro_peso': primeiro_peso, 'primeira_altura': primeira_altura, 'primeiro_IMC': primeiro_IMC, 'ultimo_peso': ultimo_peso,
       'ultima_altura': ultima_altura, 'ultimo_IMC': ultimo_IMC, 'diferenca_peso': diferenca_peso, 'coef_peso': coef_peso,
       'diferenca_tempo': diferenca_tempo, 'coef_peso_tempo': coef_peso_tempo, 'eficacia_tratamento': eficacia_tratamento}

  colunas = X_train.columns

  X_run = pd.DataFrame(columns=colunas)
  X_run = X_run.append(nova_linha, ignore_index=True)

  return clf.predict(X_run)

gradio.Interface(fn=runModel, inputs=inputs, outputs=output, title='Predição de câncer de mama').launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>